## COVID19 - Shelter Layout and Behavior
Here we will simulate a shelter with a particular layout for living spaces and a number of evacuees with daily activities. We will measure the risk of infection and find the best behavior and layout to avoid infection.

---

# 1. Run this first

#### Import Required Libraries

In [1]:
#import libraries
import numpy as np
import pandas as pd
import pylab as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
# params = {'text.usetex': True}
# plt.rcParams.update(params)
import os
import cv2
import time
import glob, os, os.path
import shutil

What if the action for RL is the decision of the  initial setting, so we can find the optimum setting based on the number of evacuees.

In [2]:
def action_set_num_of_pods(num_of_pods=40):
    global a_lay
    global b_lay
    global number_of_pods
    
    options = factors(num_of_pods)
    a_lay = np.random.choice(options,1).item()
    number_of_pods = a_lay
    
    b_lay = int(num_of_pods / a_lay)
    print(f'#of pods:{a_lay},#of pers in pod:{b_lay}')
    return a_lay,b_lay

In [3]:
def factors(x):
    l = []
    for i in range(1, x + 1):
        if x % i == 0:
            l.append(i)
    return l

In [4]:
# def action_set_num_of_pods(n_of_pods=40):
#     global number_of_pods
#     number_of_pods = n_of_pods
#     return set_layout(number_of_pods)

In [5]:
erase_case = False
case = "RLcase"
fix_random = False
number_of_evacuees = 264
number_of_days = 50
number_of_initial_infected = 1
shelter_area = 80 #m2/6m2 ==> persons (# of pods if 1 pers)

#management
do_isolate = True     #to perform isolation after a symptomatic appears
number_of_pods, capacity_in_pods = action_set_num_of_pods(shelter_area)
capacity_of_common_areas = [4,25,25] #toilet,meeting,foodcourt
# capacity_in_pods = b_lay #round(shelter_area / number_of_pods,0)

#behavioral countermeasures
rate_of_mask_use = 1.0 #rate of people using masks
disinfect_every_n_hours = 1 #to promote disinfection every n hours. Assuming it goes after 1 hour.

#of pods:5,#of pers in pod:16


In [6]:
#create case folders
if erase_case:
    if os.path.exists(f'./{case}'):
        shutil.rmtree(f'./{case}')
    os.mkdir(f'./{case}')
    os.mkdir(f'./{case}/images_ca')
    os.mkdir(f'./{case}/images_hp')
    os.mkdir(f'./{case}/images_c')
    os.mkdir(f'./{case}/schedules')
    os.mkdir(f'./{case}/densities_hp')
    os.mkdir(f'./{case}/densities_ca')
    os.mkdir(f'./{case}/logs')

In [7]:
#erase files
# filelist1 = glob.glob(os.path.join('./images_ca', "*.png"))
# filelist2 = glob.glob(os.path.join('./images_hp', "*.png"))
# filelist3 = glob.glob(os.path.join('./images_c', "*.png"))
# filelist4 = glob.glob(os.path.join('./schedules', "*.png"))
# filelist5 = glob.glob(os.path.join('./densities_hp', "*.csv"))
# filelist6 = glob.glob(os.path.join('./densities_ca', "*.csv"))
# filelist7 = glob.glob(os.path.join('./logs', "*.csv"))
# filelist8 = glob.glob(os.path.join('.', "*.avi"))
# filelist9 = glob.glob(os.path.join('.', "*.png"))
# L = filelist1+filelist2+filelist3+filelist4+filelist5+filelist6+filelist7+filelist8+filelist9
# for f in L:
#     os.remove(f)

## Define the Environment
The environment consists of **states**, **actions**, and **rewards**. States and actions are inputs for the Q-learning AI agent, while the possible actions are the AI agent's outputs.
#### States
The states in the environment are **the total number of infections detected**.  
The terminal state is when **all evacuees are infected** or **no evacuees are infected**.  
Actions are taken by the manager when an infected is detected. The simulation starts with $N_i=$ `e_number_of_infected` people.  
Also, each agent will have an awareness.

> The AI agent's goal is to learn the best way to manage the shelter by the following possible actions:
> * Enforce the use of masks by **X** percentage ==> Direct Effect: reduce probabilities.
> * Increase frequency of disinfection by **every Y hours** ==> Direct Effect reduces probability on the next hour.
> * Reduce capacity in common spaces ==> This will increase number of people in living spaces and possible infection by family contact.
> * Change the layout? ==> increase space (but what to do with people? => move to other spaces?)

#### Actions
The actions that are available to the AI agent (shelter manager) are to organize thee sheelter in a combination of these four countermeasures:
* Increase mask use
* Increase frequency of disinfection
* Reduce capacity in common spaces
* Change layout (Social distancing)

In [8]:
if fix_random:
    np.random.seed(0)
e_day = 0
e_time = 0
e_pop = number_of_evacuees #number of evacuees in shelter
e_sim_time = number_of_days #scale in days
e_rng = np.random #np.random.default_rng()
e_list_of_infected = [] #current
e_list_of_symptomatics = []
e_list_of_isolated = []
e_place_of_infection = np.zeros(4)
e_patient_zero = number_of_initial_infected
e_number_of_infected = 0 #total (incluidng isolated)
e_number_of_symptomatics = 0
e_number_of_isolated = 0
e_use_of_mask = rate_of_mask_use + 0.01 #to avoid division by zero
e_freq_disinfection = disinfect_every_n_hours #to avoid zero
e_num_of_pods = number_of_pods
e_capacity_of_common_areas = capacity_of_common_areas #toilet,meeting,foodcourt
e_density_in_common_areas = np.zeros((3,3,e_sim_time*24)) #id,num_evac,den
for i in range(e_sim_time*24):
    e_density_in_common_areas[:,0,i]= np.linspace(0,2,3).astype(int)
e_capacity_of_pods = [capacity_in_pods]*e_num_of_pods
e_density_in_pods = np.zeros((e_num_of_pods,3,e_sim_time*24)) #id,num_evac,den
for i in range(e_sim_time*24):
    e_density_in_pods[:,0,i]= np.linspace(0,e_num_of_pods-1,e_num_of_pods).astype(int)
e_max_num_fam = 0
#age,home,schedule
e_day_log = np.zeros((e_pop,26,e_sim_time)).astype(int)
#day,hour,e_number_of_infected,len(e_list_of_infected),e_number_of_symptomatics,e_number_of_isolated
e_system_log = np.zeros((e_sim_time*24,6)).astype(int)
for i in range(e_sim_time*24):
    e_system_log[i,0]= int(i)
#day,hour,home,den,mask,disinf,inf,inc,sym,out,currentloc
e_hour_log = np.zeros((e_pop,10,e_sim_time*24)).astype(int)

#define the shape of the environment (i.e., its states)
#The only things the manager can see
# state_vector = np.array([e_day,e_number_of_symptomatics,e_number_of_isolated])
# state_vector = np.array([e_day,e_number_of_infected])
state_vector = np.array([a_lay,b_lay])

#define actions
#numeric action codes: 0 = none, 1 = capacity in common areas, 2 = layout, capacity in pods
# actions = ['none', 'capacity', 'layout']
actions = ['layout']
# actions = ['none']

#Create a 3D numpy array to hold the current Q-values for each state and action pair: Q(s, a) 
#The array contains 10000 possible states (rows); for three variables describing the environment.
#The "action" dimension consists of 5 layers that will allow us to keep track of the Q-values 
#for each possible action in each state (see next cell for a description of possible actions). 
#The value of each (state, action) pair is initialized to 0.
# q_values = np.zeros((10000,len(state_vector)+len(actions))).astype(int)
# q_values[:,len(state_vector):]=-99999
#
q_values = np.full((10000,len(state_vector)+len(actions)),-99999,int)

In [9]:
# q_values[0,0:len(state_vector)]
state_vector
# q_values.shape
np.array([9,3])

array([9, 3])

#### Rewards
The last component of the environment that we need to define are the **rewards**.   
To help the AI agent learn, the current number of infected people is used as a negative reward.  
Its goal is ***to maximize its total rewards*** or ***minimize the number of infected people*** at the end of the episode.

In [10]:
#Create a 2D numpy array to hold the rewards for each state.
# rewards = (-1)*e_number_of_infected
#CREATED LATER IN TD FUNCTIONS

#### Define Helper Functions

In [11]:
#define a function that determines if the specified state is a terminal state
def is_terminal_state(verbose=False):
    if e_number_of_infected == e_number_of_isolated or e_number_of_infected == e_pop:
        if verbose: 
            print(">>>Got to a terminal state!<<<")
        return True
    else:
        return False

#define a function that will choose a random, non-terminal starting state
def get_starting_state(pop=42,sim_time=7,num_inf=1,num_pods=42,
                       cap_ca=[50,50,50],cap_pod=1,verbose=False):
    global e_rng
    global e_list_of_infected
    global e_list_of_symptomatics
    global e_list_of_isolated
    global e_place_of_infection
    global e_patient_zero
    global e_number_of_infected
    global e_number_of_symptomatics
    global e_number_of_isolated
    global e_use_of_mask
    global e_freq_disinfection
    global e_num_of_pods
    global e_capacity_of_common_areas
    global e_density_in_common_areas
    global e_capacity_of_pods
    global e_density_in_pods
    global e_max_num_fam
    global e_day
    global e_time
    global e_pop
    global e_sim_time
    global e_day_log
    global e_system_log
    global e_hour_log
    global state_vector 

    #management
    number_of_pods, capacity_in_pods = action_set_num_of_pods(shelter_area)
#     capacity_in_pods = round(shelter_area / number_of_pods,0)
    
    e_day = 0
    e_time = 0
    e_pop = pop #number of evacuees in shelter
    e_sim_time = sim_time #one week, scale in days
    e_rng = np.random #np.random.default_rng()
    e_list_of_infected = [] #current
    e_list_of_symptomatics = []
    e_list_of_isolated = []
    e_place_of_infection = np.zeros(4)
    e_patient_zero = num_inf
    e_number_of_infected = 0 #total (incluidng isolated)
    e_number_of_symptomatics = 0
    e_number_of_isolated = 0
    e_use_of_mask = rate_of_mask_use + 0.001 #to avoid division by zero
    e_freq_disinfection = disinfect_every_n_hours #to avoid zero
    e_num_of_pods = number_of_pods
    e_capacity_of_common_areas = cap_ca #toilet,meeting,foodcourt
    e_density_in_common_areas = np.zeros((3,3,e_sim_time*24)) #id,num_evac,den
    for i in range(e_sim_time*24):
        e_density_in_common_areas[:,0,i]= np.linspace(0,2,3).astype(int)
    e_capacity_of_pods = [capacity_in_pods]*e_num_of_pods
    e_density_in_pods = np.zeros((e_num_of_pods,3,e_sim_time*24)) #id,num_evac,den
    for i in range(e_sim_time*24):
        e_density_in_pods[:,0,i]= np.linspace(0,e_num_of_pods-1,e_num_of_pods).astype(int)
    e_max_num_fam = 0
    #age,home,schedule
    e_day_log = np.zeros((e_pop,26,e_sim_time)).astype(int)
    #day,hour,e_number_of_infected,len(e_list_of_infected),e_number_of_symptomatics,e_number_of_isolated
    e_system_log = np.zeros((e_sim_time*24,6)).astype(int)
    for i in range(e_sim_time*24):
        e_system_log[i,0]= int(i)
    #day,hour,home,den,mask,disinf,inf,inc,sym,out,currentloc
    e_hour_log = np.zeros((e_pop,10,e_sim_time*24)).astype(int)
    #define the shape of the environment (i.e., its states)
#     state_vector = np.array([e_day,e_number_of_symptomatics,e_number_of_isolated])
#     state_vector = np.array([e_day,e_number_of_infected])
    state_vector = np.array([a_lay,b_lay])


    reset_agents(verbose=verbose)
    create_agents(verbose=verbose)
    set_infected_agents(verbose=verbose)
    set_using_mask(verbose=verbose)
    set_agents_schedules(verbose=verbose)
    set_agents_family(verbose=verbose)


#define an epsilon greedy algorithm that will choose which action to take next 
def get_next_action_random_selection(epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
    global q_values
    init_value = -99999 #NEED  TO BE CHECKED
    #check if the state exists, a is the index of that state
    a = np.where(np.all(q_values[:,0:len(state_vector)] == state_vector,axis=1))[0]
    try:
        b = np.where(np.all(q_values == [0]*q_values.shape[1],axis=1))[0].min()
    except:
        b = -1
    
    if np.random.random() < epsilon:
        if a.size != 0:
#             print('1')
            return a,np.argmax(q_values[a.item(),len(state_vector):q_values.shape[1]]),np.max(q_values[a.item(),len(state_vector):q_values.shape[1]]),np.max(q_values[a.item(),len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print('1a')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print('1b')
            return b,np.random.randint(len(actions)),init_value,init_value
    else: #choose a random action
        r = np.random.randint(len(actions))
        if a.size != 0:
#             print('2')
            return a,r,q_values[a.item(),r+len(state_vector)],np.max(q_values[a.item(),len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print('2a')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print('2b')
            return b,r,init_value,init_value

#TRYING NOT TO SELECT RANDOM ACTIONS, INSTEAD ALWAYS NONE
#define an epsilon greedy algorithm that will choose which action to take next 
def get_next_action(epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
    global q_values
    init_value = -99999 #NEED  TO BE CHECKED
    #check if the state exists, a is the index of that state
    try:
        a = np.where(np.all(q_values[:,0:len(state_vector)] == state_vector,axis=1))[0][0]
    except:
        a = -1
    try:
        b = np.where(np.all(q_values == [0]*q_values.shape[1],axis=1))[0].min()
    except:
        b = -1
    
    if np.random.random() < epsilon:
        if a != -1:
            return a,np.argmax(q_values[a,len(state_vector):q_values.shape[1]]),np.max(q_values[a,len(state_vector):q_values.shape[1]]),np.max(q_values[a,len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print(f'1a-b:{b}')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print(f'1b-b:{b}')
            return b,np.random.randint(len(actions)),init_value,init_value
    else: #choose to do nothing
        r = 0 #np.random.randint(len(actions))
        if a != -1:
#             print('2')
            return a,r,q_values[a,r+len(state_vector)],np.max(q_values[a,len(state_vector):q_values.shape[1]])
        else:
            q_init = [init_value]*len(actions)
            if b != -1:
                q_values[b,:]=[*state_vector,*q_init]
#                 print('2a')
            else:
                q_values = np.append(q_values,[[*state_vector,*q_init]],0)
                b = q_values.shape[0]-1
#                 print('2b')
            return b,r,init_value,init_value


#define a function that will get the next behavior of agents based on the chosen action
def get_next_behavior_6(action_index):
    if actions[action_index] == 'none':
        pass
    elif actions[action_index] == 'isolate':
        ask_to_isolate_symptomatics(verbose=False)
    elif actions[action_index] == 'mask':
        ask_to_use_masks(verbose=False)
    elif actions[action_index] == 'disinfection':
        ask_to_disinfect(verbose=False)
    elif actions[action_index] == 'capacity':
        ask_to_reducecapacity(verbose=False)
    elif actions[action_index] == 'layout':
        ask_to_changelayout(verbose=False)
    return

#define a function that will get the next behavior of agents based on the chosen action
def get_next_behavior(action_index):
    if actions[action_index] == 'none':
        pass
    elif actions[action_index] == 'capacity':
        ask_to_reducecapacity(verbose=False)
    elif actions[action_index] == 'layout':
        ask_to_changelayout(verbose=False)
    return

#==========================

def reset_agents(verbose=False):
    "to destroy agents"
    if verbose: 
        print(">Reseting agents...")
    Agents.count = 0
    for evacuee in Agents.agent:
        del evacuee
    Agents.agent = []
    
def create_agents(verbose=False):
    "to create new agents"
    if verbose:
        print(f">Creating {e_pop} agents...")
    for i in range(e_pop):
        Agents(i,int(np.random.normal(40,12)))
        
def set_infected_agents(verbose=False):
    "to randomnly select number agents as infected"
    if verbose:
        print(f">Setting initial infected agents: {e_patient_zero}")
    candidates = e_rng.choice(Agents.agent,e_patient_zero)
    for evacuee in candidates:
        evacuee.a_getVirus(verbose=verbose)
        if verbose:
            print(f"{evacuee.uid}:I am patient zero")

#call every day
def write_day_log(day=0,verbose=False):
    "to save variables every day"
    global e_day_log
    for e in Agents.agent:
        e_day_log[e.uid,:,day]=[e.age,e.home,*e.schedule[:,1]]

#call every hour
def write_hour_log(day=0,hour=0,verbose=False):
    "to save variables every hour"
    global e_hour_log
    for e in Agents.agent:
        e_hour_log[e.uid,:,e_time]=[day,hour,e.home,e.mask,e.disinf,e.inf,e.inc,e.sym,e.out,e.currentloc]

#call every hour
def write_system_log(day=0,hour=0,verbose=False):
    "to save system variables"
    global e_system_log
    e_system_log[day*24+e_time,:]=[day,hour,e_number_of_infected,len(e_list_of_infected),e_number_of_symptomatics,e_number_of_isolated]
    
def set_using_mask(verbose=False):
    "to randomnly start use of mask"
    num = int(e_use_of_mask * Agents.count)
    if verbose:
        print(f">Setting {num} of evacuees with mask")
    candidates = e_rng.choice(Agents.agent,num)
    for evacuee in candidates:
        evacuee.mask = True

def set_agents_schedules(verbose=False):
    "to initiate schedules of day"
    if verbose:
        print(f">Deciding schedules for agents...")
    for evacuee in Agents.agent:
        evacuee.a_setSchedule(verbose=verbose)

def set_agents_family(verbose=False):
    "to check who is in same pod"
    if verbose:
        print(f">Verifying family members...")
    for evacuee in Agents.agent:
        evacuee.a_calculate_my_family(verbose=verbose)

#to use every hour
def calculate_densities_at_common_areas(verbose=False):
    "to calculate densities at each location"
    global e_density_in_common_areas
    if verbose:
        print(f">Calculating densities in common areas...")
    for evacuee in Agents.agent:
        if evacuee.out:
            return
        if evacuee.currentloc == 1:
            e_density_in_common_areas[0,1,e_time] += 1
        elif evacuee.currentloc == 2:
            e_density_in_common_areas[1,1,e_time] += 1
        elif evacuee.currentloc == 3:
            e_density_in_common_areas[2,1,e_time] += 1
        else:
            pass
    e_density_in_common_areas[:,2,e_time] = e_density_in_common_areas[:,1,e_time] / e_capacity_of_common_areas[:]

#to use every hour
def calculate_densities_at_home_pods(verbose=False):
    "to ask agents to calculate density at home if currentloc is home"
    global e_density_in_pods
    if verbose:
        print(f">Calculating densities at home pods...")
    for evacuee in Agents.agent:
        if evacuee.out:
            return
        evacuee.a_calculate_my_density_at_home(verbose=verbose)
    e_density_in_pods[:,2,e_time] = e_density_in_pods[:,1,e_time] / e_capacity_of_pods
    
#run every hour (step)
def set_next_location(verbose=False):
    "to set the next event of each evacuee"
    if verbose:
        print(f">Setting next location...")
    for evacuee in Agents.agent:
        if evacuee.out:
            evacuee.nextloc = -1
        else:
            evacuee.a_pick_next_loc(e_time)

def move_agents_to_next_location(verbose=False):
    "to move all agent to its new location"
    if verbose:
        print(f">Moving agents to next location...")
    for evacuee in Agents.agent:
        if evacuee.out:
            evacuee.currentloc = -1
            evacuee.nextloc = -1
            return
        evacuee.a_move(verbose=verbose)

#at the end of day
def update_infected(verbose=False):
    "to update list of infected"
    global e_list_of_infected
    global e_number_of_infected
    if verbose:
        print(f">Updating infected list and incubation days...")
    for evacuee in e_list_of_infected:
        if not evacuee.inf:
            if verbose:
                print(f"Removing evacuee {evacuee.uid} with inf:{evacuee.inf}")
            e_list_of_infected.remove(evacuee)
            e_number_of_infected -=1
        else:
            evacuee.inc -= 1
            if verbose:
                print(f"@{evacuee.uid} remaining days:{evacuee.inc}")
            if evacuee.inc == 0:
                evacuee.a_symptoms()
    
    e_list_of_infected = list(set(e_list_of_infected))
    

def calculate_virus_spreading_in_common_areas(verbose=False):
    "to spread virus based on probabilities and location"
    if verbose:
        print(f">Calculating virus spread in common areas...")
    num_per_area = [0,0,0] #toilet,meeting,foodcourt
    for evacuee in e_list_of_infected:
        if evacuee.out:
            return
        if evacuee.currentloc == 0:
            return
        if evacuee.currentloc == 1: #toilet
            num_per_area[0] += 1
        if evacuee.currentloc == 2: #meeting
            num_per_area[1] += 1
        if evacuee.currentloc == 3: #foodcourt
            num_per_area[2] += 1
    
    for evacuee in Agents.agent:
        if evacuee.out or evacuee.inf:
            return
        evacuee.a_check_infection_risk_in_common_areas(num_per_area,verbose=verbose)

def calculate_virus_spreading_at_home_pods(verbose=False):
    "to spread virus based on probabilities at homes"
    if verbose:
        print(f">Calculating virus spread at homes...")
    for evacuee in e_list_of_infected:
        if evacuee.out:
            return
        if evacuee.currentloc == 0 and len(evacuee.fam) > 0:
            n_inf_mem = 1
            n_mem = len(evacuee.fam)
            for member in evacuee.fam:
                if member.inf and not member.out:
                    n_inf_mem +=1
            for member in evacuee.fam:
                if not member.inf and not member.out:
                    member.a_check_infection_risk_at_home(n_inf_mem,verbose=verbose)

def apply_disinfection(verbose=False):
    for evacuee in Agents.agent:
        evacuee.a_disinfect_apply(verbose=verbose)

def remove_disinfection(verbose=False):
    for evacuee in Agents.agent:
        evacuee.a_disinfect_remove(verbose=verbose)
        
#move clock one hour
def step(verbose=False):
    "to move one point in time"
#     global e_time
#     e_time += 1
    if verbose:
        print("="*10)

#### Functions for agents

In [12]:
class Agents(object):
    """ An agent class"""
    count = 0
    agent = []
    def __init__(self, uid, a,verbose=False):
        "Initiate agent variables"
        self.uid = self.count
        self.__class__.count +=1
        self.age = a
        self.home = np.random.choice(e_num_of_pods,1).item()
        self.fam = []
        self.mask = False
        self.disinf = False
        self.inf = False
        self.inc = np.random.choice(np.linspace(2,14).astype(int),1).item() #days
        self.sym = False
        self.out = False
        self.currentloc = 0 #(0)home; (1)toilet; (2)meeting; (3)foodcourt
        self.schedule = np.zeros((24,2)).astype(int) #list of list [time event]
        self.nextloc = 0 #code of next event (0)home; (1)toilet; (2)meeting; (3)foodcourt
        Agents.agent.append(self)
                    
    def a_check_infection_risk_in_common_areas(self,num_of_infected,verbose=False):
        "to check risk based on age and punished by conditions"
        p = 0
        #basic susceptibility based on age
        if self.age < 20:
            p = 0.00006
        elif self.age >= 20 and self.age < 30:
            p=0.0003
        elif self.age >= 30 and self.age < 40:
            p=0.0008
        elif self.age >= 40 and self.age < 50:
            p=0.0015
        elif self.age >= 50 and self.age < 60:
            p=0.006
        elif self.age >= 60 and self.age < 70:
            p=0.022
        elif self.age >= 70 and self.age < 80:
            p=0.051
        else:
            p=0.093
        
        #increased or reduction based on conditions
        if self.mask:
            p = 0.5*p
        
        #based on disinfection
        if self.disinf:
            p = 0.5*p
        
        #based on location
        if self.currentloc != 0:
            p = num_of_infected[self.currentloc - 1]*p 
            #last is best since if no one infected nearby then p=0, 
            #otherwise increases with number of infected people
        
        #based on density
        p  = e_density_in_common_areas[self.currentloc - 1,2,e_day*24+e_time]*p
        
        #get infected based on probability p
        if np.random.random() < p:
            if verbose:
                print(f"@{self.uid} my prob was {p:0.2f}")
            self.a_getVirus(verbose=verbose)

    def a_check_infection_risk_at_home(self,num_of_infected,verbose=False):
        "to check risk based on age and punished by conditions"
        p = 0
        #basic susceptibility based on age
        if self.age < 20:
            p = 0.00006
        elif self.age >= 20 and self.age < 30:
            p=0.0003
        elif self.age >= 30 and self.age < 40:
            p=0.0008
        elif self.age >= 40 and self.age < 50:
            p=0.0015
        elif self.age >= 50 and self.age < 60:
            p=0.006
        elif self.age >= 60 and self.age < 70:
            p=0.022
        elif self.age >= 70 and self.age < 80:
            p=0.051
        else:
            p=0.093
        
        #increased or reduction based on conditions
        if self.mask:
            p = 0.5*p
        
        #based on disinfection
        if self.disinf:
            p = 0.5*p

        #based on location
        p = num_of_infected*p 
        
        #based on density
        p  = e_density_in_pods[self.home,1,e_day*24+e_time]*p
        
        #get infected based on probability p
        if np.random.random() < p:
            if verbose:
                print(f"@{self.uid} my prob was {p:0.2f}")
            self.a_getVirus(verbose=verbose)
    
    def a_calculate_my_family(self,verbose=False):
        "to calculate who is my family - same pod"
        global e_max_num_fam
        for evacuee in Agents.agent:
            if evacuee is not self:
                if evacuee.home == self.home:
                    self.fam.append(evacuee)
        #update max num of family
        if len(self.fam) > e_max_num_fam:
                e_max_num_fam = len(self.fam)
        if verbose:
            print(f"@{self.uid} has {len(self.fam)} members at #{self.home} pod")

    def a_calculate_my_density_at_home(self,verbose=False):
        "to calculate the density in a pod"
        if self.out:
            return
        if self.currentloc == 0:
            e_density_in_pods[self.home,1,e_time]+=1
    
    def a_getVirus(self,verbose=False):
        "to get infected"
        global e_list_of_infected
        global e_number_of_infected
        global e_place_of_infection
        self.inf = True
        e_list_of_infected.append(self)
        e_number_of_infected += 1
        e_place_of_infection[self.currentloc] += 1
        if verbose:
            print(f"@{self.uid} infected at {self.currentloc}")
    
    def a_setSchedule(self,verbose=False):
        "to set today's schedule"
        if self.out:
            return
        #structure of schedule
        #7-8,12-14,18-19 breakfast (fix) ==> locations 0 or 3 (home or foodcourt)
        #9-11,15-17,20-22 random number of activities (max 3 actions) (toilet, meeting, home)
        self.schedule[:,1]=np.zeros((24,), dtype=int) #set default home
        self.schedule[:,0]=np.linspace(0,23,24,dtype=int) #set hours of day
        #meals
        c = e_rng.choice([7,8],1,p=[0.5,0.5]).item() #random time for breakfast
        self.schedule[c,1]= e_rng.choice([0,3],1,p=[0.5,0.5]).item()
        c = e_rng.choice([12,13,14],1,p=[0.4,0.3,0.3]).item() #random time for lunch
        self.schedule[c,1]= e_rng.choice([0,3],1,p=[0.5,0.5]).item()
        c = e_rng.choice([18,19],1,p=[0.5,0.5]).item() #random time for dinner
        self.schedule[c,1]= e_rng.choice([0,3],1,p=[0.5,0.5]).item()
        #other activities
        slots = [[9,10,11],[15,16,17],[20,21,22]]
        for slot in slots:
            c = e_rng.choice(slot,1,p=[1./3.,1./3.,1./3.]).item() #random time for toilet
            self.schedule[c,1]= e_rng.choice([0,1,2],1,p=[1./3.,1./3.,1./3.]).item() #maybe toilet
            slot.remove(c)
            a0 = e_rng.choice([0,2],1,p=[0.5,0.5])
            a1 = e_rng.choice([0,2],1,p=[0.5,0.5])
            self.schedule[slot[0],1]=a0
            self.schedule[slot[1],1]=a1
        if verbose:
            pass#print(f"@{self.uid}:schedule set")#{self.schedule}")
    
    def a_pick_next_loc(self,e_time,verbose=False):
        "to pick the corresponding event at time e_time"
        self.nextloc = self.schedule[e_time,1]
    
    def a_move(self,verbose=False):
        "to move to next activity"
        self.currentloc = self.nextloc
    
    def a_symptoms(self,verbose=False):
        "to show symptoms"
        global e_list_of_symptomatics
        global e_number_of_symptomatics
        self.sym = True
        e_list_of_symptomatics.append(self)
        e_number_of_symptomatics += 1
        if verbose:
            print(f"@{self.uid} showing symptomps!")
        
    def a_isolate(self,verbose=False):
        "to take out of simulation"
        global e_list_of_infected
        global e_list_of_isolated
        global e_number_of_isolated
        global e_list_of_symptomatics
        global e_number_of_symptomatics
        self.out = True
        self.inc = -1
        self.currentloc = -1
        self.schedule[:,1] = [-1]*24
        e_list_of_infected.remove(self)
        e_list_of_symptomatics.remove(self)
        e_number_of_symptomatics -= 1
        e_list_of_isolated.append(self)
        e_number_of_isolated += 1
        if verbose:
            print(f"@{self.uid} isolated!")
        
    def a_disinfect_apply(self,verbose=False):
        self.disinf = True
            
    def a_disinfect_remove(self,verbose=False):
        self.disinf = False

#### Functions to adjust evacuee behavior (asked by manager)

In [13]:
def ask_to_isolate_symptomatics(verbose=False):
    global e_list_of_infected
    global e_number_of_infected
    if verbose:
        print(f"!Asking symptomatics to isolate")
    for evacuee in Agents.agent:
        if evacuee.inf and evacuee.sym and not evacuee.out:
            evacuee.a_isolate(verbose=verbose)       

def ask_to_use_masks(verbose=False):
    "Increasing on p=20% the use of masks"
    global e_use_of_mask
    
    p = 0.2 #<=== include later as parameter
    if verbose:
        print(f"!Asking to increase use of masks")
    #list up people not using masks
    no_mask = [evacuee for evacuee in Agents.agent if not evacuee.mask]
    rate = len(no_mask) / e_pop
    if rate < p:
        #all
        candidates = no_mask
        e_use_of_mask += rate
        if verbose:
            print(f">(a) Setting additional {rate}({len(no_mask)}) evacuees with mask")
    else:
        #some
        num = int(p*len(no_mask))
        candidates = e_rng.choice(no_mask,num)
        e_use_of_mask += p
        if verbose:
            print(f">Setting additional {p*100}%({num}) evacuees with mask")
    
    for evacuee in candidates:
        evacuee.mask = True

def ask_to_disinfect(verbose=False):
    "when disinfecting frequency is reduced"
    global e_freq_disinfection
    e_freq_disinfection -= 1
    if e_freq_disinfection == 0:
        e_freq_disinfection = 1

def ask_to_reducecapacity(verbose=False):
    "to reduce capacity of common areas"
    global e_capacity_of_common_areas
    e_capacity_of_common_areas = list(np.array(e_capacity_of_common_areas)-1)
    e_capacity_of_common_areas = [1 if i < 1 else i for i in e_capacity_of_common_areas ]
    
def ask_to_changelayout(verbose=False):
    "to increase capacity of pods"
    global e_capacity_of_pods
    
    e_capacity_of_pods = list(np.array(e_capacity_of_pods)-1)

#### Functions to plot outputs

In [14]:
def analyze_data(verbose=False):
    dfA = pd.read_csv("daylog.csv")
    dfB = pd.read_csv("hourlog.csv")
    dfC = pd.read_csv("systemlog.csv")
    return dfA, dfB, dfC

def plot_histogram():
    #plot age histogram
    mu, sigma = 40, 12
    count, bins, ignored = plt.hist(e_day_log[:,0,0],30,density=True)
    plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
                   np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
             linewidth=2, color='r')
    plt.xlim(0,90)
    plt.xlabel("age")
    plt.ylabel("frequency")
    plt.title("Histogram of Population age")
    plt.text(10,0.03,r"$\mathcal{N}(40,12)$",fontsize=12)
    plt.savefig(f"./histogram.png",dpi=150,facecolor="w",transparent=False)
    plt.close('all')

def plot_home_pods(day=e_day,hour=e_time,cmap="Reds",gridcolor='black'):
    #plot homes
    fig, ax = plt.subplots(figsize=(14,12))
    a = e_density_in_pods[:,2,e_time]
#     a = a.reshape(5,4) #<==== hard coded :(
    m = np.random.choice(factors(number_of_pods),1).item()
    n = int(number_of_pods / m)
    a = a.reshape(m,n)
    plt.imshow(a,cmap=cmap)
    
    #Mayor ticks
    ax.set_xticks(np.arange(-.5, 4, 1), minor=False)
    ax.set_yticks(np.arange(-.5, 5, 1), minor=False)
    ax.grid(which='major', color=gridcolor, linestyle='-', linewidth=6)
    # Minor ticks
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 5, 1), minor=True)
    ax.grid(which='minor', color=gridcolor, linestyle='-', linewidth=6)
    
    #hide labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    # Loop over data dimensions and create text annotations.
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            text = ax.text(j, i, a[i, j], fontsize=20,backgroundcolor="white",
                           ha="center", va="center", color="black")
    ax.set_title("Situation of home pods",fontsize=20)
    
    fig.text(0.45,0.1,f"DAY:{day} - TIME:{e_time}:00",fontsize=20,ha="left")
        
    t = 24*day+hour
    plt.savefig(f"./images_hp/homepod{t:04}.png",dpi=150,facecolor="w",transparent=False)
    fig = None
    ax = None
    plt.close('all')

def plot_common_areas(day=e_day,hour=e_time,cmap="Reds",gridcolor="black"):
    #plot places
    fig, ax =  plt.subplots()
    b = e_density_in_common_areas[:,2,e_time]
    b = b.reshape(1,3)
    plt.imshow(b,cmap=cmap)
    
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    #Mayor ticks
    ax.set_xticks(np.arange(-.5, 3, 1), minor=False)
    ax.set_yticks(np.arange(-.5, 1, 1), minor=False)
    ax.grid(which='major', color=gridcolor, linestyle='-', linewidth=6)
    
    #Text in places
    ax.text(0,0.5,"Toilet",fontsize=14,backgroundcolor="white",ha="center")
    ax.text(1,0.5,"Meeting room",fontsize=14,backgroundcolor="white",ha="center")
    ax.text(2,0.5,"Foodcourt",fontsize=14,backgroundcolor="white",ha="center")
    # Loop over data dimensions and create text annotations.
    for i in range(b.shape[0]):
        for j in range(b.shape[1]):
            text = ax.text(j, i, b[i, j], fontsize=20,backgroundcolor="white",
                           ha="center", va="center", color="black")
    ax.set_title("Situation of service areas")
    
    fig.text(0.4,0.15,f"DAY:{day} - TIME:{e_time}:00",fontsize=10,ha="left")
    
    t = 24*day+hour
    plt.savefig(f"./images_ca/commonarea{t:04}.png",dpi=150,facecolor="w",transparent=False)
    fig = None
    ax = None
    plt.close('all')

def plot_curves(day=e_day,hour=e_time):
    if day*24+hour == 0:
        return "Snapshot 0 is ignored"
    #plot system results
    fig, ax =  plt.subplots()

    t= day*24+hour
    ax.plot(e_system_log[:t,2],label="total infected",ls='-',c='r')
    ax.plot(e_system_log[:t,3],label="current infected",ls=':',c='orange')
    ax.plot(e_system_log[:t,4],label="symptomatics",ls='-.',c='m')
    ax.plot(e_system_log[:t,5],label="isolated",ls='--',c='k')
    ymax = e_system_log[:t,2].max() * 1.05
    ax.set_ylim(0,ymax)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%0.0f'))
    ax.set_xlabel("time (hr)")
    ax.set_ylabel("Population")
    ax.set_title("Summary of shelter situation")
    ax.legend(loc='best')

    fig.text(0.4,0.15,f"DAY:{day} - TIME:{e_time}:00",fontsize=10,ha="left")
    
    plt.savefig(f"./images_c/curves{t:04}.png",dpi=150,facecolor="w",transparent=False)
    fig = None
    ax = None
    plt.close('all')
    
def get_snapshot(day=e_day,e_time=e_time):
    plot_home_pods(day,e_time)
    plot_common_areas(day,e_time)
    plot_curves(day,e_time)

def plot_schedules(day=e_day,verbose=False):
    fig, (ax1,ax2) = plt.subplots(1,2,gridspec_kw={'width_ratios': [35, 1]},figsize=(10,6))

    s = e_day_log[:,2:,day]
    s = np.rot90(s,k=3)
    
    cmap = mpl.colors.ListedColormap(['white','green','blue','red'])
    bounds=[0,0.9,1.9,2.9,3.9]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    ax1.imshow(s,cmap=cmap, norm=norm,aspect='auto',origin="lower")
    ax1.tick_params(axis='x',labelsize=12)
    ax1.tick_params(axis='y',labelsize=12)
    #Mayor ticks
    ax1.set_xticks(np.arange(-.5, 100, 1), minor=True)
    ax1.set_yticks(np.arange(-.5, 23, 1), minor=True)
    ax1.grid(which='minor', color='grey', linestyle='-', linewidth=1)
    ax1.set_xticks(np.linspace(0,100,11).astype(int))
    ax1.set_yticks(np.linspace(0,23,24).astype(int))
    ax1.set_xlabel("Evacuee id",fontsize=12)
    ax1.set_ylabel("time (hr)",fontsize=12)
    ax1.grid(False)
    # make a color bar
    cb = mpl.colorbar.ColorbarBase(ax2,cmap=cmap,norm=norm,ticks=[0.5,1.5,2.5,3.5],orientation='vertical')
    cb.set_ticklabels(["Home","Toilet","Meeting","Food"])
    cb.ax.tick_params(labelsize='large')
    
    fig.text(0.0,0.01,f"DAY:{day}",fontsize=20,backgroundcolor="white",ha="left")
        
    name = f"./schedules/{day:04}.png"
    fig.savefig(name,dpi=150)
    fig = None
    ax1 = None
    ax2 = None
    plt.close('all')

In [15]:
def makeVideo(image_folder='images_c',video_name = 'video_c.avi',fps=10,verbose=False):
    images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc('M','J','P','G'), fps, (width,height))
    for image in images:
        if verbose:
            print(image)
        video.write(cv2.imread(os.path.join(image_folder, image)))
    cv2.destroyAllWindows()
    video.release()

## The Model

In [16]:
def setup(pop=132,sim_time=50,num_inf=1,num_pods=20,cap_ca=[4,25,25],
          cap_pod=4,files=False,plots=False,verbose=False):
    get_starting_state(pop=pop,sim_time=sim_time,num_inf=num_inf,num_pods=num_pods,
                       cap_ca=cap_ca,cap_pod=cap_pod,verbose=verbose)
    write_day_log(0,verbose=verbose)
    if files:
        np.savetxt(f"./logs/daylog-0000.csv",e_day_log[:,:,e_time],delimiter=',',
               header="age,home,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23",comments='',fmt='%26d')
    if plots:
        plot_histogram()

#setup(e_pop=132,e_sim_time=50,num_inf=1,num_pods=20,cap_ca=[4,25,25],
 #         cap_pod=4,verbose=False):

In [17]:
def go(files=False,plots=False,videos=False,verbose=False):
    global e_day
    global e_time
    t0 = time.time()
    for i in range(e_sim_time):
        e_day = i
        set_agents_schedules(verbose=verbose)
        for j in range(24):
            e_time = j
            if verbose:
                print(f"Day {e_day}, {e_time} hours")
            if e_freq_disinfection != 0:
                if e_time % e_freq_disinfection == 0: #every 1-6 hours (compulsory disinfection at beginning of day)
                    apply_disinfection(verbose=verbose)
            set_next_location(verbose=verbose)
            move_agents_to_next_location(verbose=verbose)
            calculate_densities_at_common_areas(verbose=verbose)
            calculate_densities_at_home_pods(verbose=verbose)
            calculate_virus_spreading_in_common_areas(verbose=verbose)
            calculate_virus_spreading_at_home_pods(verbose=verbose)
            remove_disinfection(verbose=verbose)
            write_hour_log(e_day,e_time,verbose=verbose)
            write_system_log(e_day,e_time,verbose=verbose)
            if files:
                np.savetxt(f"./densities_hp/den{24*e_day+e_time:04}.csv",e_density_in_pods[:,:,e_time],delimiter=',',comments='',
                           fmt=['%d','%d','%0.2f'])
                np.savetxt(f"./densities_ca/den{24*e_day+e_time:04}.csv",e_density_in_common_areas[:,:,e_time],delimiter=',',
                           comments='',fmt=['%0.2f','%0.2f','%0.2f'])
                np.savetxt(f"./logs/hourlog{24*e_day+e_time:04}.csv",e_hour_log[:,:,e_time],delimiter=',',
                           header="day,hour,home,mask,disinf,inf,inc,sym,out,currentloc",comments='',fmt='%10d')
                np.savetxt(f"./logs/systemlog{24*e_day+e_time:04}.csv",e_system_log[:,:],delimiter=',',
                           header="day,hour,Tinf,Cinf,sym,isol",comments='',fmt='%6d')
            if plots:
                get_snapshot(e_day,e_time)
            step(verbose=verbose)
        update_infected(verbose=verbose)
        write_day_log(i,verbose=verbose)
        if files:
            np.savetxt(f"./logs/daylog{e_day:04}.csv",e_day_log[:,:,i],delimiter=',',
                   header="age,home,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23",comments='',fmt='%26d')
        if plots:
            plot_schedules(i,verbose=verbose)
        if verbose:
            print("*"*25)
            print(f"Total of infected evacs:{e_number_of_infected}")
            print(f"Current infected (today):{len(e_list_of_infected)}")
            print(f"Current isolated (symp):{e_number_of_infected - len(e_list_of_infected)}")
            print("*"*25)
            print("*"*25)
            print(f"Total of infected evacs:{e_number_of_infected}")
            print(f"Current infected (today):{len(e_list_of_infected)}")
            print(f"Current symptomatics (symp):{e_number_of_symptomatics}")
            print(f"Current isolated (symp):{e_number_of_isolated}")
            print("*"*25)
        #evaluate and perform countermeasures for next day
        if do_isolate:
            ask_to_isolate_symptomatics(verbose=verbose)
        #ask_to_disinfect(verbose=verbose)
        #ask_to_use_masks(verbose=verbose)
        #ask_to_reducecapacity(verbose=verbose)
        #ask_to_changelayout(verbose=verbose)
        e_time =  0 #restart day clock
        if is_terminal_state(verbose=verbose):
            break
#     if plots:
#         get_snapshot(e_sim_time,23)
    if videos:
        makeVideo(image_folder=f'./{case}/schedules',video_name = f'./{case}/schedules.avi',fps=1,verbose=verbose)
        makeVideo(image_folder=f'./{case}/images_c',video_name = f'./{case}/curves.avi',fps=5,verbose=verbose)
        makeVideo(image_folder=f'./{case}/images_ca',video_name = f'./{case}/commonareas.avi',fps=5,verbose=verbose)
        makeVideo(image_folder=f'./{case}/images_hp',video_name = f'./{case}/homepods.avi',fps=5,verbose=verbose)
    t1 = time.time()
    print(f"Finished:{t1-t0} s")
    return e_place_of_infection

# go()
#go(verbose=True) #only text
#go(plots=True,videos=False,verbose=False) #only plots
# go(plots=True,videos=True,verbose=False) #plots and videos
# go(plots=True,videos=True,verbose=True) #all

setup(pop=250,sim_time=50,num_inf=1,num_pods=20,cap_ca=[4,25,25],
          cap_pod=4,files=False,plots=False,verbose=False)

go(files=False,plots=False,videos=False,verbose=False)

---

### Tests at laptop (2021.02.08)
This is the model without RL but stops before 30 days if needed.

* Test1: 30 days,  100 people, plots and videos and verbose = 760.038s
* Test2: 30 days,  100 people, plots and videos, no verbose = 777.289s
* Test3: 30 days,  100 people, ONLY plots                   = 672.856s 
* Test4: 30 days,  100 people, ONLY verbose                 =  10.137s
* Test5: 30 days,  100 people, NOTHING                      =   7.410s

### Tests at laptop (2021.01.26)
This is the model without RL.

* Test1: 7 days,  100 people, plots and videos and verbose = 195.45s
* Test2: 7 days,  100 people, plots and videos, no verbose = 182.90s
* Test3: 7 days,  100 people, ONLY plots                   = 165.57s 
* Test4: 7 days,  100 people, ONLY verbose                 =   3.08s
* Test5: 7 days,  100 people, NOTHING                      =   2.53s


# possible questions  

* Does pods with larger families get infected first?
* Does families of patient zero get infected first?
* Does increasing the capacity of pods (size) instead of separation between pods affects results?

# Ideas

Layout can be also how you put families.
*  Clusterize the shelter (different comon spaces)
*  Set families together and individuals closer.

## Train the Model
Our next task is for our AI agent to learn about its environment by implementing a Q-learning model. The learning process will follow these steps:
1. Choose a random, non-terminal state for the agent to begin this new episode.
2. Let evacuees move and infect others during a whole day (or an hour?).
3. Calculate the state vector at the end of the day (or the end of the hour?).
4. Choose an action for the current state. Actions will be chosen using an *epsilon greedy algorithm*. This algorithm will usually choose the most promising action for the AI agent, but it will occasionally choose a less promising option in order to encourage the agent to explore the environment.
5. Perform the chosen action for the next day, and transition to the next state (i.e., next day schedules).
6. Receive the reward for moving to the new state, and calculate the temporal difference.
7. Update the Q-value for the previous state and action pair.
8. If the new (current) state is a terminal state, go to #1. Else, go to #2.

This entire process will be repeated across 1000 episodes. This will provide the AI agent sufficient opportunity to learn the dynamics of the system.

#### QL support functions

In [18]:
def update_state_vector():
    global state_vector
#     state_vector = np.array([e_day,e_number_of_symptomatics,e_number_of_isolated])
#     state_vector = np.array([e_day,e_number_of_infected])
#     state_vector = np.array([a_lay,b_lay,e_day,e_number_of_infected])
    state_vector = np.array([a_lay,b_lay])

#### Train the AI Agent using Q-Learning

In [19]:
def setup_TD(pop=100,sim_time=30,num_inf=1,num_pods=20,cap_ca=[4,25,25],cap_pod=4,files=False,plots=False,verbose=False):
    get_starting_state(pop=pop,sim_time=sim_time,num_inf=num_inf,num_pods=num_pods,cap_ca=cap_ca,
                       cap_pod=cap_pod,verbose=verbose)

In [20]:
def go_TD(action_record,duration,state_record,
          iterations=100,episode=0,epsilon=0.9,files=False,plots=False,videos=False,verbose=False):
    global e_day
    global e_time
    
#     t0 = time.time()
    for i in range(e_sim_time):
#         print(i)
        e_day = i
        set_agents_schedules(verbose=verbose)
        for j in range(24):
            e_time = j
            if e_freq_disinfection != 0:
                if e_time % e_freq_disinfection == 0: #every 1-6 hours (compulsory disinfection at beginning of day)
                    apply_disinfection(verbose=verbose)
            set_next_location(verbose=verbose)
            move_agents_to_next_location(verbose=verbose)
            calculate_densities_at_common_areas(verbose=verbose)
            calculate_densities_at_home_pods(verbose=verbose)
            calculate_virus_spreading_in_common_areas(verbose=verbose)
            calculate_virus_spreading_at_home_pods(verbose=verbose)
            remove_disinfection(verbose=verbose)
            #added on March 1
            if do_isolate:
                ask_to_isolate_symptomatics(verbose=verbose)
            #Q-Learning (mode 2)
            #update_state_vector()
            #===================
            step(verbose=verbose)
        
        update_infected(verbose=verbose)
        
#         #Q-Learning (mode 1)
#         update_state_vector()
#         state_record[episode,e_day,:]=state_vector
#         #======================
#         #evaluate and perform countermeasures for next day
#         state_index,action_index,old_q_value,max_q_value = get_next_action(epsilon)
#         get_next_behavior(action_index)
#         #receive the reward for moving to the new state, and calculate the temporal difference
#         reward = -1000*e_number_of_infected#symptomatics
#         temporal_difference = reward + (discount_factor * max_q_value) - old_q_value

#         #update the Q-value for the previous state and action pair
#         new_q_value = old_q_value + (learning_rate * temporal_difference)
#         #print(q_values.shape,state_index.item(),action_index,q_values[state_index,5+action_index])
#         q_values[state_index,len(state_vector)+action_index] = new_q_value
        
        e_time =  0 #restart day clock
#         if episode == iterations - 1:
#         action_record[episode,e_day] = action_index
        if is_terminal_state(verbose=verbose):
            break
#     t1 = time.time()
#     print(f"Finished:{t1-t0} s")
    #print(f'Days:{e_day}')
    #Q-Learning (mode 3) - MonteCarlo
    update_state_vector()
#     state_record[episode,e_day,:]=state_vector
    state_record[episode,:]=state_vector
    #======================
    #evaluate and perform countermeasures for next day
    state_index,action_index,old_q_value,max_q_value = get_next_action(epsilon)
    get_next_behavior(action_index)
    #receive the reward for moving to the new state, and calculate the temporal difference
    reward = -1000*e_number_of_infected-10*e_day#symptomatics
    temporal_difference = reward + (discount_factor * max_q_value) - old_q_value
    #update the Q-value for the previous state and action pair
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    #print(q_values.shape,state_index.item(),action_index,q_values[state_index,5+action_index])
    q_values[state_index,len(state_vector)+action_index] = new_q_value
    action_record[episode,e_day] = action_index
    duration[episode,:]=[e_number_of_infected,e_day]
    place_record[episode,:]=e_place_of_infection

# 2. Run this latest

In [21]:
t0 = time.time()
print(q_values)
#define training parameters
epsilon = 0.9 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #discount factor for future rewards (0 is myopic, 1 is future interest)
learning_rate = 0.9 #the rate at which the AI agent should learn (0 is no learning)
iterations = 1000

global duration
global action_record
global state_record

duration = np.zeros((iterations,2)) #episode rows to store days
action_record = np.zeros((iterations,e_sim_time))
# state_record = np.zeros((iterations,e_sim_time,len(state_vector)))
state_record = np.zeros((iterations,len(state_vector)))
place_record = np.zeros((iterations,4)) #iteration rows and  4 places columns
                        
#run through 1000 training episodes
for episode in range(iterations):
    print(f">>>Episode:{episode}")
    setup_TD(pop=number_of_evacuees,sim_time=number_of_days,num_inf=number_of_initial_infected,
             num_pods=number_of_pods,cap_ca=capacity_of_common_areas,cap_pod=capacity_in_pods,
             files=False,plots=False,verbose=False)
    go_TD(action_record,duration,state_record,iterations,episode,epsilon,files=False,plots=False,
          videos=False,verbose=False)
print('Training complete!')
t1 = time.time()
print(f"Finished {iterations} iterations in {t1-t0} s")

[[-99999 -99999 -99999]
 [-99999 -99999 -99999]
 [-99999 -99999 -99999]
 ...
 [-99999 -99999 -99999]
 [-99999 -99999 -99999]
 [-99999 -99999 -99999]]
>>>Episode:0
#of pods:5,#of pers in pod:16
>>>Episode:1
#of pods:40,#of pers in pod:2
>>>Episode:2
#of pods:2,#of pers in pod:40
>>>Episode:3
#of pods:10,#of pers in pod:8
>>>Episode:4
#of pods:16,#of pers in pod:5
>>>Episode:5
#of pods:16,#of pers in pod:5
>>>Episode:6
#of pods:1,#of pers in pod:80
>>>Episode:7
#of pods:1,#of pers in pod:80
>>>Episode:8
#of pods:8,#of pers in pod:10
>>>Episode:9
#of pods:40,#of pers in pod:2
>>>Episode:10
#of pods:20,#of pers in pod:4
>>>Episode:11
#of pods:20,#of pers in pod:4
>>>Episode:12
#of pods:2,#of pers in pod:40
>>>Episode:13
#of pods:16,#of pers in pod:5
>>>Episode:14
#of pods:5,#of pers in pod:16
>>>Episode:15
#of pods:5,#of pers in pod:16
>>>Episode:16
#of pods:80,#of pers in pod:1
>>>Episode:17
#of pods:10,#of pers in pod:8
>>>Episode:18
#of pods:40,#of pers in pod:2
>>>Episode:19
#of pods:2

>>>Episode:182
#of pods:5,#of pers in pod:16
>>>Episode:183
#of pods:1,#of pers in pod:80
>>>Episode:184
#of pods:8,#of pers in pod:10
>>>Episode:185
#of pods:40,#of pers in pod:2
>>>Episode:186
#of pods:20,#of pers in pod:4
>>>Episode:187
#of pods:10,#of pers in pod:8
>>>Episode:188
#of pods:16,#of pers in pod:5
>>>Episode:189
#of pods:10,#of pers in pod:8
>>>Episode:190
#of pods:8,#of pers in pod:10
>>>Episode:191
#of pods:40,#of pers in pod:2
>>>Episode:192
#of pods:40,#of pers in pod:2
>>>Episode:193
#of pods:80,#of pers in pod:1
>>>Episode:194
#of pods:2,#of pers in pod:40
>>>Episode:195
#of pods:20,#of pers in pod:4
>>>Episode:196
#of pods:10,#of pers in pod:8
>>>Episode:197
#of pods:10,#of pers in pod:8
>>>Episode:198
#of pods:10,#of pers in pod:8
>>>Episode:199
#of pods:10,#of pers in pod:8
>>>Episode:200
#of pods:1,#of pers in pod:80
>>>Episode:201
#of pods:10,#of pers in pod:8
>>>Episode:202
#of pods:16,#of pers in pod:5
>>>Episode:203
#of pods:20,#of pers in pod:4
>>>Episode

>>>Episode:365
#of pods:40,#of pers in pod:2
>>>Episode:366
#of pods:2,#of pers in pod:40
>>>Episode:367
#of pods:2,#of pers in pod:40
>>>Episode:368
#of pods:5,#of pers in pod:16
>>>Episode:369
#of pods:1,#of pers in pod:80
>>>Episode:370
#of pods:4,#of pers in pod:20
>>>Episode:371
#of pods:16,#of pers in pod:5
>>>Episode:372
#of pods:20,#of pers in pod:4
>>>Episode:373
#of pods:8,#of pers in pod:10
>>>Episode:374
#of pods:40,#of pers in pod:2
>>>Episode:375
#of pods:20,#of pers in pod:4
>>>Episode:376
#of pods:1,#of pers in pod:80
>>>Episode:377
#of pods:80,#of pers in pod:1
>>>Episode:378
#of pods:40,#of pers in pod:2
>>>Episode:379
#of pods:5,#of pers in pod:16
>>>Episode:380
#of pods:10,#of pers in pod:8
>>>Episode:381
#of pods:20,#of pers in pod:4
>>>Episode:382
#of pods:16,#of pers in pod:5
>>>Episode:383
#of pods:1,#of pers in pod:80
>>>Episode:384
#of pods:4,#of pers in pod:20
>>>Episode:385
#of pods:80,#of pers in pod:1
>>>Episode:386
#of pods:20,#of pers in pod:4
>>>Episode

>>>Episode:548
#of pods:80,#of pers in pod:1
>>>Episode:549
#of pods:8,#of pers in pod:10
>>>Episode:550
#of pods:40,#of pers in pod:2
>>>Episode:551
#of pods:10,#of pers in pod:8
>>>Episode:552
#of pods:16,#of pers in pod:5
>>>Episode:553
#of pods:1,#of pers in pod:80
>>>Episode:554
#of pods:8,#of pers in pod:10
>>>Episode:555
#of pods:4,#of pers in pod:20
>>>Episode:556
#of pods:10,#of pers in pod:8
>>>Episode:557
#of pods:1,#of pers in pod:80
>>>Episode:558
#of pods:80,#of pers in pod:1
>>>Episode:559
#of pods:20,#of pers in pod:4
>>>Episode:560
#of pods:1,#of pers in pod:80
>>>Episode:561
#of pods:10,#of pers in pod:8
>>>Episode:562
#of pods:5,#of pers in pod:16
>>>Episode:563
#of pods:10,#of pers in pod:8
>>>Episode:564
#of pods:2,#of pers in pod:40
>>>Episode:565
#of pods:80,#of pers in pod:1
>>>Episode:566
#of pods:4,#of pers in pod:20
>>>Episode:567
#of pods:80,#of pers in pod:1
>>>Episode:568
#of pods:10,#of pers in pod:8
>>>Episode:569
#of pods:5,#of pers in pod:16
>>>Episode

>>>Episode:731
#of pods:5,#of pers in pod:16
>>>Episode:732
#of pods:10,#of pers in pod:8
>>>Episode:733
#of pods:10,#of pers in pod:8
>>>Episode:734
#of pods:1,#of pers in pod:80
>>>Episode:735
#of pods:80,#of pers in pod:1
>>>Episode:736
#of pods:5,#of pers in pod:16
>>>Episode:737
#of pods:1,#of pers in pod:80
>>>Episode:738
#of pods:16,#of pers in pod:5
>>>Episode:739
#of pods:2,#of pers in pod:40
>>>Episode:740
#of pods:2,#of pers in pod:40
>>>Episode:741
#of pods:5,#of pers in pod:16
>>>Episode:742
#of pods:16,#of pers in pod:5
>>>Episode:743
#of pods:1,#of pers in pod:80
>>>Episode:744
#of pods:4,#of pers in pod:20
>>>Episode:745
#of pods:10,#of pers in pod:8
>>>Episode:746
#of pods:10,#of pers in pod:8
>>>Episode:747
#of pods:16,#of pers in pod:5
>>>Episode:748
#of pods:10,#of pers in pod:8
>>>Episode:749
#of pods:10,#of pers in pod:8
>>>Episode:750
#of pods:1,#of pers in pod:80
>>>Episode:751
#of pods:40,#of pers in pod:2
>>>Episode:752
#of pods:80,#of pers in pod:1
>>>Episode

>>>Episode:914
#of pods:1,#of pers in pod:80
>>>Episode:915
#of pods:2,#of pers in pod:40
>>>Episode:916
#of pods:8,#of pers in pod:10
>>>Episode:917
#of pods:80,#of pers in pod:1
>>>Episode:918
#of pods:40,#of pers in pod:2
>>>Episode:919
#of pods:20,#of pers in pod:4
>>>Episode:920
#of pods:40,#of pers in pod:2
>>>Episode:921
#of pods:8,#of pers in pod:10
>>>Episode:922
#of pods:4,#of pers in pod:20
>>>Episode:923
#of pods:16,#of pers in pod:5
>>>Episode:924
#of pods:8,#of pers in pod:10
>>>Episode:925
#of pods:40,#of pers in pod:2
>>>Episode:926
#of pods:10,#of pers in pod:8
>>>Episode:927
#of pods:4,#of pers in pod:20
>>>Episode:928
#of pods:1,#of pers in pod:80
>>>Episode:929
#of pods:5,#of pers in pod:16
>>>Episode:930
#of pods:1,#of pers in pod:80
>>>Episode:931
#of pods:5,#of pers in pod:16
>>>Episode:932
#of pods:10,#of pers in pod:8
>>>Episode:933
#of pods:4,#of pers in pod:20
>>>Episode:934
#of pods:4,#of pers in pod:20
>>>Episode:935
#of pods:5,#of pers in pod:16
>>>Episode

In [22]:
np.savetxt(f"./qvalues1112.csv",q_values[:,:],delimiter=',',header="a,b,aLAY",
           comments='',fmt=['%d','%d','%d'])
qout = pd.read_csv('./qvalues1112.csv')
qout = qout[qout['a']!=-99999]
qout['aLAY'].max()

-23556

In [23]:
qout.sort_values(by='aLAY',ascending=False)

,a,b,aLAY
10008,80,1,-23556
10001,40,2,-45566
10007,20,4,-115820
10004,16,5,-196931
10003,10,8,-289567
10006,8,10,-320314
10000,5,16,-525120
10009,4,20,-645205
10002,2,40,-1325334
10005,1,80,-2639825


# This is output analysis

In [ ]:
duration.transpose()[1].mean()
duration.transpose()[0].mean()/132

In [ ]:
duration

In [ ]:
#choose iteration to plot
i = 16
i_days = int(duration[i,1])
print(i_days)
fig = plt.figure()
fig.patch.set_facecolor('white')
ax = fig.add_subplot(111)
# ax.plot(state_record[i,:i_days,0],state_record[i,:i_days,1],label='infected',color=(1,0,0))
ax.plot(state_record[i,:i_days,0],state_record[i,:i_days,1],label='symptomatics',color=(0,0.4,0),zorder=1)
# ax.scatter(state_record[:,0],state_record[:,3],color=(0,0.4,0),zorder=1)
ax.plot(state_record[i,:i_days,0],state_record[i,:i_days,2],label='isolated',color=(0,0,1),linestyle='--')
ax.legend(loc=(1.2,0.8))
# plt.ylim(0,5)
plt.ylabel("number")

# ax2 = ax.twinx()
# ax2.scatter(np.linspace(0,i_days-1,i_days),action_record[i,:i_days],zorder=0)
# ax2.plot(action_record[i,:i_days],'--',label='actions',zorder=0)
# ax2.set_yticks([0,1,2])
# ax2.set_yticklabels(["none","capacity","layout"])

plt.xlabel("days")
# plt.ylabel("actions")

In [ ]:
sum_of_rows = place_record.sum(axis=1)
normalized_array = place_record / sum_of_rows[:, np.newaxis]

In [ ]:
sizes = [ normalized_array.transpose()[0].mean(),
normalized_array.transpose()[1].mean(),
normalized_array.transpose()[2].mean(),
normalized_array.transpose()[3].mean()]

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Home', 'Toilet', 'Meeting Room', 'Food Court'
sizes = sizes
explode = (0.1, 0, 0, 0)  # only "explode" the 1st slice (i.e. 'Home')

fig, ax = plt.subplots()
ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90,normalize=False)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
# for j in range(place_record.shape[0]):
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     ax.bar([0,1,2,3],place_record[j])
#     ax.set_xticks([0,1,2,3])
#     ax.set_xticklabels(["home","toilet","meeting room","food court"])
#     plt.ylim(0,place_record.max())
#     plt.savefig(f"./figures/{j:04d}.png")
#     plt.close()

In [ ]:
action_record

In [ ]:
i=16
fig = plt.figure()
plt.scatter(np.linspace(0,len(action_record[i])-1,len(action_record[i])),action_record[i])
plt.plot(action_record[i],'--')
plt.xlabel("days")
plt.ylabel("action")
plt.yticks([0,1,2],labels=["none","capacity","layout"])
fig.patch.set_facecolor('white')

In [ ]:
fig = plt.figure()
# plt.scatter(np.linspace(0,iterations-1,iterations),duration[:,1]) #0 infected, 1 days
plt.plot(duration[:,0],'--')
plt.xlabel("iteration")
plt.ylabel("#infected")
fig.patch.set_facecolor('white')

In [ ]:
dat00 = [duration[:x,0].mean() for x in range(1,len(duration[:,0])-1)]
dat01 = [duration[:x,0].mean()+duration[:x,0].std() for x in range(1,len(duration[:,0])-1)]
dat02 = [duration[:x,0].mean()-duration[:x,0].std() for x in range(1,len(duration[:,0])-1)]

dat1 = [duration[:x,1].mean() for x in range(1,len(duration[:,1])-1)]
dat2 = [duration[:x,1].mean()+duration[:x,1].std() for x in range(1,len(duration[:,1])-1)]
dat3 = [duration[:x,1].mean()-duration[:x,1].std() for x in range(1,len(duration[:,1])-1)]

In [ ]:
fig = plt.figure()
plt.plot(dat00)
plt.plot(dat01)
plt.plot(dat02)
plt.ylim(0,20)
plt.xlabel("iteration")
plt.ylabel("#infected")
fig.patch.set_facecolor('white')

In [ ]:
fig = plt.figure()
plt.plot(dat1)
plt.plot(dat2)
plt.plot(dat3)
plt.ylim(0,30)
plt.xlabel("iteration")
plt.ylabel("days")
fig.patch.set_facecolor('white')

In [ ]:
np.savetxt(f"./qvaluesRL10.csv",q_values[:,:],delimiter=',',header="day,symp,iso,aNONE,aCAP,aLAY",
           comments='',fmt=['%d','%d','%d','%.3f','%.3f','%.3f'])

In [ ]:
np.savetxt(f"./qvalues1111.csv",q_values[:,:],delimiter=',',header="a,b,aLAY",
           comments='',fmt=['%d','%d','%d'])

In [ ]:
#[e_day,e_time,e_number_of_infected,e_number_of_symptomatics,e_number_of_isolated]
# np.set_printoptions(precision=3)
# dfq = pd.DataFrame(q_values,columns=["day","symp","iso","aNONE","aCAP","aLAY"])
dfq = pd.DataFrame(q_values,columns=["day","inf","aLAY"])
dfq.head(100)

# Making sense of the outputs

In [ ]:
qout = pd.read_csv('./qvalues1111.csv')
qout = qout[qout['a']!=-99999]
qout

In [ ]:
qout['aLAY'].max()

In [ ]:
number_of_pods

In [ ]:
#this is the first row with no data
m = qout.loc[(qout['aNONE'] == -99999.0) & (qout['aCAP'] == -99999.0) & (qout['aLAY'] == -99999.0)].index[0]
m

In [ ]:
qout.iloc[0:m,:]

In [ ]:
# outdata = qout.iloc[0:m,:]
outdata = qout

In [ ]:
outdata.shape

In [ ]:
x = outdata["day"]

In [ ]:
y2 = outdata["inf"]
# y3 = outdata["iso"]

In [ ]:
cdict = {"aNONE":0,"aCAP":1,"aLAY":2}
z = list(map(lambda x: cdict[x], outdata.iloc[:,3:].idxmax(axis=1)))

# Idea 2

* Try to plot frequencies of actions based on date to see which action is more preferable based on timing.
* Then plot state versus time

In [ ]:
test2 = pd.DataFrame([])
test2["time"]=x
test2["action"]=z

# test2["inf"]=y1
test2["symp"]=y2
test2["iso"]=y3

test2.sort_values(by=['time'],inplace=True)
test3 = test2.copy()
test4 = test2.copy()
test5 = test2.copy()

dfgb2 = test2.groupby(["time"])["action"].apply(pd.Series.mode).to_frame()
# dfgb3 = test3.groupby(["time"])["inf"].agg(pd.Series.max).to_frame()
dfgb4 = test4.groupby(["time"])["symp"].agg(pd.Series.mean).to_frame()
dfgb4max = test4.groupby(["time"])["symp"].agg(pd.Series.max).to_frame()
dfgb4min = test4.groupby(["time"])["symp"].agg(pd.Series.min).to_frame()
dfgb5 = test5.groupby(["time"])["iso"].agg(pd.Series.mean).to_frame()


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)

# ax.plot(dfgb3.index,dfgb3["inf"],color=(0,0,1),label='inf')
ax.plot(dfgb4.index,dfgb4["symp"],color=(0,1,0),label='symp')
ax.plot(dfgb4max.index,dfgb4max["symp"],color=(0.5,0.5,0.5),label='symp.max')
ax.plot(dfgb4min.index,dfgb4min["symp"],color=(0.5,0.5,0.5),label='symp.min')
ax.plot(dfgb5.index,dfgb5["iso"],color=(0,0,0),label='iso')
ax2 = ax.twinx()
ax2.scatter(dfgb2.index.get_level_values(0),
            dfgb2["action"],color=(1,0,0),label='action')

ax.set_xlabel('time (day)')
# ax.set_ylabel('inf')
ax.set_ylabel('symp')
ax.set_ylabel('iso')
ax.set_ylabel('symp/iso')
ax2.set_ylabel('action')
ax2.set_yticks([0,1,2])
ax2.set_yticklabels(["none","capacity","layout"])
# ax.set_ylim(0,max(dfgb4["symp"])+1)
# ax2.set_ylim(0,3)
ax.legend(loc='upper center')
ax2.legend(loc='upper left')
# plt.savefig('1000.png')